# Supplementary Information: Holmes *et al.* 2017

# 6. Treatment effect

In [ ]:
%pylab inline

import numpy as np
import pandas as pd
import pystan
import scipy
import seaborn as sns

import tools

import warnings
warnings.filterwarnings('ignore')

sns.set_context('notebook')

## Building the model

Our partially-pooled model has constrained the linear fits of measured *output* intensity to *input* intensity, such that the relationships are drawn from a group (*probe ID*) level distribution of parameters. We have seen that there is some variability in the extent to which those parameters are well-estimated, but we have not yet taken into account the effect of applying passage treatment.

To do this, we extend the model by introducing the coefficient $\gamma$, to represent a global influence due to the passage step being included in the experiment, and the covariate $t_i$ as a 0/1 value that indicates whether the probe $i$ was measured as a *control* (0) or *treatment* (1) experiment. The value $\gamma t_i$ is an offset for the linear relationship, and does not reflect an additional influence on the slope describing the relationship between *input* and *output* (i.e how much an *input* is modified to give the *output*).

We therefore construct the following model of the experiment:

$$y_i = \alpha_{j[i]} + \beta_{j[i]} x_i + \gamma t_i + \epsilon_i$$
$$\alpha_{j[i]} \sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$$
$$\beta_{j[i]} \sim N(\mu_{\beta}, \sigma_{\beta}^2)$$
$$\sigma_{\alpha} \sim U(0, 100)$$
$$\sigma_{\beta} \sim U(0, 100)$$

* $y_i$: measured intensity output on the array for probe $i$ (specific to each replicate)
* $x_i$: measured intensity input on the array for probe $i$ (specific to each replicate)
* $t_i$: 0/1 indicating whether the probe $i$ was measured in a control (0) or treatment (1) experiment
* $\alpha_{j[i]}$: the linear intercept, this is a constant 'offset' for all *output* measurements relative to *input* measurements, but it differs for each *probe ID*; this is drawn from a Normal distribution $\sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$
* $\mu_{\alpha}$: the mean offset for all *probe ID*s
* $\sigma_{\alpha}$: the standard deviation of the offset for all *probe ID*s - sampled from a Uniform distribution $U(0, 100)$
* $\beta_{j[i]}$: the linear slope, this is the relative change in measured intensity between *input* and *output* intensities *at the probe ID level* - it differs for each *probe ID*; this is drawn from a Normal distribution $\sim N(\mu_{\beta}, \sigma_{\beta}^2)$
* $\mu_{\beta}$: the mean slope for all *probe ID*s
* $\sigma_{\beta}$: the standard deviation of the slope for all *probe ID*s - sampled from a Uniform distribution $U(0, 100)$
* $\gamma$: a global estimate of the influence on *output* measured intensity of introducing passage (*treatment*) into the experiment
* $\epsilon_i$: error in the model prediction for probe $i$

### Stan model construction and fit

We need to define `data`, `parameters` and our `model` for `Stan`.

In the `data` block, we have:

* `N`: `int`, the number of data points)
* `J`: `int`, the number of unique probe IDs (`J` < `N`)
* `probe`: `int[N]`, an index list of probe identities - one index representing six probe measurements (i.e. three control, three treatment) - there are `J` probes
* `t`: `vector[N]`, 0/1 control/treatment values for each probe
* `x`: `vector[N]`, the input log(intensity) values
* `y`: `vector[N]`, the output log(intensity) values

In the `parameter` block, we have:

* `a`: `real vector[J]`, representative input log(intensity)
* `mu_a`: `real`, an unconstrained value to be fit that represents the mean offset for each probe ID
* `b`: `real vector[J]`, effect on log(intensity) of passing through the experiment, specific to a probe ID
* `mu_b`: `real`, an unconstrained value to be fit that represents the mean slope for each probe ID
* `g`: `real`, estimate of the influence of treatment on the output measured intensity
* `sigma`: `real<lower=0>`, the error in the prediction
* `sigma_a`: `real<lower=0,upper=100>`, standard deviation of the offset per probe ID
* `sigma_b`: `real<lower=0,upper=100>`, standard deviation of the slope per probe ID

We also define a `transformed parameter`:

* `y_hat[i] <- b[probe[i]] * x[i] + a[probe[i]] + g * t[i]`: the linear relationship describing $\hat{y}$, our estimate of experimental output intensity, which is subject to variance `sigma`.

We define the model as:

$$\sigma_{\alpha} \sim U(0, 100)$$
$$\alpha_{j[i]} \sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$$
$$\sigma_{\beta} \sim U(0, 100)$$
$$\beta_{j[i]} \sim N(\mu_{\beta}, \sigma_{\beta}^2)$$
$$y \sim N(\hat{y}, \sigma^2)$$


In [ ]:
# load clean, normalised, indexed data
# data = pd.read_csv("output/normalised_array_data.tab", sep="\t")  # full dataset
data = pd.read_csv("output/reduced_normalised_indexed_array_data.tab", sep="\t")  # reduced dataset for test

# useful values
probe_ids = data['probe'].unique()
nprobes = len(probe_ids)

In [ ]:
# define unpooled stan model
treatment_model = """
data {
  int<lower=0> N;
  int<lower=0> J;
  int<lower=1, upper=J> probe[N];
  vector[N] t;
  vector[N] x;
  vector[N] y;
}
parameters {
  vector[J] a;
  vector[J] b;
  real g;
  real mu_a;
  real mu_b;
  real<lower=0> sigma;
  real<lower=0,upper=100> sigma_a;
  real<lower=0,upper=100> sigma_b;
}
transformed parameters{
  vector[N] y_hat;

  for (i in 1:N)
    y_hat[i] = a[probe[i]] + b[probe[i]] * x[i] + g * t[i];
}
model {
  sigma_a ~ uniform(0, 100);
  a ~ normal(mu_a, sigma_a);

  sigma_b ~ uniform(0, 100);
  b ~ normal(mu_b, sigma_b);

y ~ normal(y_hat, sigma);
}
"""

In [ ]:
# relate python variables to stan variables
treatment_data_dict = {'N': len(data),
                       'J': nprobes,
                       'probe': data['probe_index'] + 1,
                       't': data['treatment'],
                       'x': data['input'],
                       'y': data['output']}

In [ ]:
# run stan fit
treatment_fit = pystan.stan(model_code=treatment_model,
                            data=treatment_data_dict,
                            iter=1000, chains=2)

## Inspecting the fit

We're now interested in how the 'base' fit of the relationship between *input* and *output* is modified when we take a treatment effect offset into account, but also in the size of this effect.

In [ ]:
# Get fits to alpha, beta, gamma by probe ID
alpha_estimates = pd.Series(treatment_fit['a'].mean(0), index=probe_ids)
alpha_se = pd.Series(treatment_fit['a'].std(0), index=probe_ids)
beta_estimates = pd.Series(treatment_fit['b'].mean(0), index=probe_ids)
beta_se = pd.Series(treatment_fit['b'].std(0), index=probe_ids)
gamma_mean = treatment_fit['g'].mean(0)
gamma_se = treatment_fit['g'].std(0)

In [ ]:
# Inspect the estimates
alpha_estimates.head()

In [ ]:
# Inspect the estimates
beta_estimates.head()

In [ ]:
# Inspect the estimates
print(gamma_mean, tools.print_intervals(treatment_fit, 'g', 50))

There is an estimate that the offset $\gamma$ due to the treatment effect is negative, and the 50% CI does not include zero. The mean estimates for $\beta_{j[i]}$ appear to be in a similar range, though the estimates of $\alpha_{j[i]}$ (and corresponding errors in this estimate) have increased to counter the effect of introducing $\gamma$.

Introducing $\gamma$ has not greatly affected the distribution of estimates for $\beta_{j[i]}$.

### Intercept $\alpha_{j[i]}$

In [ ]:
# Plot means distribution
g = sns.boxplot(alpha_estimates)
g.set_title("Distribution of mean intercept")
g.set_xlabel("mean(alpha)");

In [ ]:
# Plot means distribution
g = sns.boxplot(alpha_se)
g.set_title("Distribution of intercept standard error")
g.set_xlabel("se(alpha)");

In [ ]:
# errors for intercept outliers
tools.plot_threshold_errors(alpha_estimates, alpha_se, 65)

### Slopes $\beta_{j[i]}$

In [ ]:
# Plot means distribution
g = sns.boxplot(beta_estimates)
g.set_title("Distribution of mean intercept")
g.set_xlabel("mean(beta)");

In [ ]:
# Plot error distribution
g = sns.boxplot(beta_se)
g.set_title("Distribution of standard error")
g.set_xlabel("se(beta)");

In [ ]:
# errors for intercept outliers
tools.plot_threshold_errors(beta_estimates, beta_se, 0.85)

In [ ]:
# errors for intercept outliers
tools.plot_threshold_errors(beta_estimates, beta_se, 0.7, upper=False)

In [ ]:
# errors for intercept outliers
tools.plot_threshold_errors(beta_estimates, beta_se, 2, upper=False)